# <center> Get user graph for `Russian`

In [38]:
import sys
import util
import json
import time
from twython import Twython

### Twitter API credentials

In [39]:
APP_KEY = "AV4KujYNS1y6HdmQ4zXxHw2Tx"
APP_SECRET = "xd2FuFaS5zE1ON407C3U5X0kdEtRJIjH9OMDmgyMmxQCZMFlli"
OAUTH_TOKEN = "1058011325316636672-HkpzhyiOHdJkuwi3FD9QrkMFbuHuVt"
OAUTH_TOKEN_SECRET = "iiD75H9ERtkATnd74pVRBfM7TI189BSmDasYzN2uUO123"

### Retrieve user graph

Get user set.

In [3]:
users = list()
for line in open("../Datasets/russian_election2018_rtu.txt"):
    line = line.split()
    users.append(int(line[2]))
    users.append(int(line[3]))
# users = util.get_authors("../Datasets/russian_rtid.txt")

Get the followgraph of users present in the trace.

In [ ]:
# init
twitter = Twython(APP_KEY, APP_SECRET, OAUTH_TOKEN, OAUTH_TOKEN_SECRET)
FollowGraph = dict()
outfile = "../Datasets/russian_adjList.txt"

for i,u in enumerate(users):
    
    # reinit cursor
    cursor = -1

    # while there are followers left for this user
    while cursor != 0:
        sys.stdout.write("User {} / {}...\r".format(i, len(users)))

        try:
            # get followers
            result = twitter.get_followers_ids(id=u, cursor=cursor)
            ids = result['ids']
            cursor = result['next_cursor']

            # write to out
            with open(outfile, 'a') as out:
                for v in ids:
                    out.write("{} {}\n".format(u,v))

            # wait if API rate limit has been reached
            calls_remaining = int(twitter.get_lastfunction_header('x-rate-limit-remaining'))
            if calls_remaining == 0:
                time_to_wait = 15*60
                start = time.time()
                elapsed_time = 0
                while elapsed_time < time_to_wait:
                    sys.stdout.flush()
                    sys.stdout.write("User {} / {}... Rate limit reached... {} seconds to wait.\r".format(i, len(users), time_to_wait - elapsed_time))
                    time.sleep(1)
                    elapsed_time = time.time() - start
                    
        except:
            cursor += 1
            continue
            
                    
#         except:
#             cursor += 1
#             continue
                    
                
#             # twitter api rate limit exceeded --> sleep then reconnect and retry
#             except:
#                 waiting_time = float(twitter.get_lastfunction_header('x-rate-limit-reset'))
# #                 del twitter
#                 start = time.time()
#                 elapsed_time = time.time() - start
#                 while elapsed_time <= waiting_time:
#                     sys.stdout.flush()
#                     sys.stdout.write("Elapsed time : {} on {} total time to wait.\n".format(elapsed_time, waiting_time))
# #                     sys.stdout.write("Sleeping... {:.0f}min {:.0f}sec left\r".format(time_left//60, time_left%60))
#                     time.sleep(1)
#                     elapsed_time = time.time() - start
# #                 twitter = Twython(APP_KEY, APP_SECRET, OAUTH_TOKEN, OAUTH_TOKEN_SECRET)
#                 retry = True

In [11]:
int(twitter.get_lastfunction_header('x-rate-limit-reset'))

1563219410

In [8]:
twitter = Twython(APP_KEY, APP_SECRET, OAUTH_TOKEN, OAUTH_TOKEN_SECRET)
twitter.show_status(id=1150759441941368832)
twitter.get_lastfunction_header('x-rate-limit-remaining')

'898'

## VERIF

Get followgraph from .txt file.

In [48]:
followgraph = dict()
for line in open("../Datasets/russian_adjList.txt"):
    lead, follow = int(line.split()[0]), int(line.split()[1])
    if lead in followgraph:
        followgraph[lead].add(follow)
    else:
        followgraph[lead] = {follow}

In [49]:
for line in open("../Datasets/russian_adjList2.txt"):
    lead, follow = int(line.split()[0]), int(line.split()[1])
    if lead in followgraph:
        followgraph[lead].add(follow)
    else:
        followgraph[lead] = {follow}

In [50]:
with open("../Datasets/russian_adjList_full.txt", "w") as out:
    for lead in followgraph:
        for follow in followgraph[lead]:
            out.write("{} {}\n".format(lead, follow))

22451

problematic users

In [40]:
users = set()
for line in open("../Datasets/russian_getfollowers_error_logs.txt"):
    line = line.split()
    if int(line[1]) != -1:
        users.add(int(line[0]))

In [41]:
len(users)

21

re-crawl problematic users

In [43]:
followgraph = dict()
twitter = Twython(APP_KEY, APP_SECRET, OAUTH_TOKEN, OAUTH_TOKEN_SECRET)
for i,u in enumerate(users):
    followgraph[u] = set()
    
    # reinit cursor
    cursor = -1
    
    # while there are followers left for this user
    while cursor != 0:
        
        try:
            # get followers
            result = twitter.get_followers_ids(id=u, cursor=cursor)
            sys.stdout.flush()
            sys.stdout.write("User {}: {} / {} ok...\r".format(u, i+1, len(users)))

            # write to out
            for v in result['ids']:
                followgraph[u].add(v)
            cursor = result['next_cursor']

            # wait if API rate limit has been reached
            calls_remaining = int(twitter.get_lastfunction_header('x-rate-limit-remaining'))
            if calls_remaining == 0:
                time_to_wait = 15*60 + 5 # add 5 seconds just in case
                start = time.time()
                elapsed_time = 0
                while elapsed_time < time_to_wait:
                    sys.stdout.flush()
                    sys.stdout.write("User {} / {}... Rate limit reached... {} seconds to wait.\r".format(i+1, len(users), round(time_to_wait - elapsed_time)))
                    time.sleep(1)
                    elapsed_time = time.time() - start
                    
        except:
            sys.stdout.flush()
            sys.stdout.write("User {} / {} ERROR...\r".format(i, len(users)))
            cursor += 1
            continue

In [46]:
out = open("../Datasets/russian_adjList2.txt", "w")
for u in users:
    for v in followgraph[u]:
        out.write("{} {}\n".format(u,v))
out.close()

In [26]:
result = twitter.get_followers_ids(id=423660053)

In [36]:
twitter.get_lastfunction_header('x-rate-limit-remaining')

'879'

division by zero


In [51]:
users_ok = set()
for line in open("../Datasets/russian_adjList_full.txt"):
    users_ok.add(int(line.split()[0]))

In [52]:
users = set()
for line in open("../Datasets/russian_getfollowers_error_logs.txt"):
    users.add(int(line.split()[0]))

In [54]:
users.intersection(users_ok)

{28859692,
 33879930,
 43429477,
 99659322,
 147655301,
 171309893,
 190583229,
 194023516,
 201677046,
 209735575,
 218700419,
 239467085,
 241844789,
 255862660,
 267628080,
 291229200,
 300747334,
 348074426,
 374280605,
 387840841,
 388116313,
 405054531,
 410091837,
 423660053,
 430022947,
 432671140,
 454994048,
 456686553,
 463814103,
 484722237,
 492487068,
 542310868,
 576380779,
 591557393,
 712320296,
 980583553,
 1030831236,
 1103455836,
 1369309567,
 1372916954,
 1457061193,
 1588856568,
 1590944965,
 1660986140,
 1852407247,
 1919030486,
 2169624355,
 2321897600,
 2375337679,
 2436883562,
 2558040008,
 2574896866,
 2592819421,
 2603552835,
 2607984906,
 2653850772,
 2742997421,
 2752356549,
 2828540888,
 2832071262,
 2873392763,
 2907348474,
 2950679629,
 2955196012,
 2971895205,
 3024839255,
 3026202434,
 3029837242,
 3080357232,
 3172137337,
 3293820729,
 3300279694,
 3377034477,
 3923911037,
 4201675241,
 4363056442,
 4550216505,
 4895431835,
 715550092233687040,
 7841